In [ ]:
import os
import shutil
import subprocess
import time

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
print(rdkit.__version__)
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
import py3Dmol

from confworks.util import calc_rmsd_single_mol
from confworks.util import optimize_molecule, read_multiconf_sdf, get_boltzmann_weights

PATH = os.getcwd()

## Read molecule first

In [ ]:
# Lets read our dataset of molecules as pandas dataframe
file_path = os.path.join(PATH, 'data', 'molecules.csv')
dataset = pd.read_csv(file_path)

mol_list = []

# Lets draw all molecules and their trivial names on a single image
for i, row in dataset.iterrows():
    smiles = row['smiles']
    name = row['name']
    if name == 'Quinine':
        mol = Chem.MolFromSmiles(smiles)
        mol.SetProp("_Name", name)
        mol_list.append(mol)

legends = [mol.GetProp('_Name') for mol in mol_list]
Draw.MolsToGridImage(mol_list, molsPerRow= 1, subImgSize=(300,300), legends=legends, returnPNG=False)

## Generate 3D conformers for a given molecule using RDKit's ETKDG method

In [ ]:
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
import py3Dmol

from confworks.util import calc_rmsd_matrix_single_mol

# this is a spin off idea i wantred to test
quinine_params = AllChem.EmbedParameters()
quinine_params.randomSeed = 42
quinine_params.numThreads = 4
quinine_params.timeout = 100
quinine_params.clearConfs = True
quinine_params.onlyHeavyAtomsForRMS = True
quinine_params.useRandomCoords = True
quinine_params.enforceChirality = True
quinine_params.pruneRmsThresh = 0.1

quinine_NUM_CONFS = 3

quinine = mol_list[0]
quinine_3D = Chem.AddHs(quinine)
AllChem.EmbedMultipleConfs(quinine_3D, quinine_NUM_CONFS, quinine_params)
print(quinine_3D.GetNumConformers())
IPythonConsole.drawMol3D(quinine_3D, confId=-1)


## Calculate single point energies of all conformers using xTB

In [ ]:
from confworks.util import xtb_SP
from confworks.util import conformer_search
quinine_rdkit = xtb_SP(quinine_3D)
print(min([conf.GetDoubleProp('conf_energy')for conf in quinine_rdkit.GetConformers()]))


In [ ]:
from confworks.util import save_multiconf_sdf
quinine_rdkit_filepath = os.path.join(PATH, 'data', 'quinine_rdkit.sdf')
save_multiconf_sdf(quinine_rdkit, quinine_rdkit_filepath)

In [ ]:
from confworks.util import read_multiconf_sdf
quinine_rdkit_filepath = os.path.join(PATH, 'data', 'quinine_rdkit.sdf')
quinine_rdkit = read_multiconf_sdf(quinine_rdkit_filepath)
IPythonConsole.drawMol3D(quinine_rdkit, confId=-1)

In [ ]:
quinine_xtb, _ = optimize_molecule(quinine_rdkit, gfn_xtb='ff', opt_lvl='extreme')

quinine_xtb_filepath = os.path.join(PATH, 'data', 'quinine_xtb.sdf')
save_multiconf_sdf(quinine_xtb, quinine_xtb_filepath)
print(min([conf.GetDoubleProp('conf_energy')for conf in quinine_xtb.GetConformers()]))


In [ ]:
import re
from rdkit import Chem
from rdkit.Geometry import Point3D
import pandas as pd

def parse_xtb_trajectory_log(log_file_path):
    """
    Parses an xtb trajectory log file to extract iteration-wise energies,
    gradient norms, and XYZ coordinates. This version is tailored for
    logs where each frame starts with atom count, then a comment line with
    energy/gnorm, followed by coordinates.

    Args:
        log_file_path (str): Path to the xtb log file.

    Returns:
        list: A list of dictionaries, where each dictionary contains:
              'energy': float,
              'gnorm': float,
              'xyz_coords': list of lists (atom symbol, x, y, z)
    """
    frames_data = []

    # Pattern to detect the number of atoms line
    num_atoms_pattern = re.compile(r"^\s*(\d+)\s*$")

    # Pattern to extract energy and gnorm from the comment line
    comment_line_pattern = re.compile(
        r"^\s*energy:\s*(-?\d+\.\d+)\s+"
        r"gnorm:\s*(\d+\.\d+)\s+"
        r"xtb:\s*\d+\.\d+\.\d+\s+\(\S+\)\s*$"
    )

    # Pattern to parse atom lines (Symbol X Y Z)
    atom_line_pattern = re.compile(r"^\s*([A-Za-z]{1,3})\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\s*$")

    with open(log_file_path, 'r') as f:
        lines = f.readlines()

    line_iter = iter(lines)

    while True:
        try:
            # 1. Read the number of atoms line
            num_atoms_line = next(line_iter).strip()
            match_num_atoms = num_atoms_pattern.match(num_atoms_line)

            if not match_num_atoms:
                break # End of file or unexpected content

            num_atoms = int(match_num_atoms.group(1))

            # 2. Read the comment line (energy, gnorm, xtb version)
            comment_line = next(line_iter).strip()
            match_comment = comment_line_pattern.match(comment_line)

            if not match_comment:
                print(f"Warning: Expected comment line after atom count, but found: '{comment_line}'")
                break

            energy = float(match_comment.group(1))
            gnorm = float(match_comment.group(2))

            # 3. Read atom coordinates
            xyz_coords = []
            for _ in range(num_atoms):
                atom_line = next(line_iter).strip()
                match_atom_line = atom_line_pattern.match(atom_line)
                if match_atom_line:
                    symbol = match_atom_line.group(1)
                    x = float(match_atom_line.group(2))
                    y = float(match_atom_line.group(3))
                    z = float(match_atom_line.group(4))
                    xyz_coords.append([symbol, x, y, z])
                else:
                    print(f"Warning: Could not parse atom line: '{atom_line}'")
                    # Attempt to skip lines if parsing fails to try and recover
                    # For a robust parser, more sophisticated error handling might be needed
                    break
            
            # If we successfully parsed all atom lines for this frame
            if len(xyz_coords) == num_atoms:
                frames_data.append({
                    'energy': energy,
                    'gnorm': gnorm,
                    'xyz_coords': xyz_coords
                })

        except StopIteration:
            break # Reached end of file
        except Exception as e:
            print(f"An error occurred during parsing: {e} at line: {line_iter.gi_frame.f_lineno}")
            break

    return frames_data


def add_xtb_conformers_to_mol(mol_template: Chem.Mol, log_file_path: str) -> Chem.Mol:
    """
    Reads an xtb trajectory log file, extracts geometries and associated data,
    and adds each as a new conformer to the provided RDKit Mol object.

    Args:
        mol_template (Chem.Mol): An RDKit Mol object (e.g., the initial geometry,
                                  or just the molecular graph). It should have the
                                  correct number of atoms.
        log_file_path (str): Path to the xtb optimization log file.

    Returns:
        Chem.Mol: The original Mol object with new conformers added, each
                  containing 'Step', 'Energy', and 'Gradient Norm' properties.
    """
    
    # Ensure the template mol has no conformers initially if you want to
    # populate it purely from the log, or ensure it has at least one for atom count.
    # We will remove existing conformers to start fresh.
    mol = Chem.Mol(mol_template) # Create a copy to avoid modifying the original
    mol.RemoveAllConformers()

    parsed_data = parse_xtb_trajectory_log(log_file_path)

    if not parsed_data:
        print(f"No optimization data found in {log_file_path}. Returning original mol.")
        return mol_template # Return the original mol if no data

    for step_idx, frame in enumerate(parsed_data):
        energy = frame['energy']
        gnorm = frame['gnorm']
        xyz_coords = frame['xyz_coords']
        
        # Check if the number of atoms matches the template molecule
        if len(xyz_coords) != mol.GetNumAtoms():
            print(f"Warning: Number of atoms in log frame {step_idx+1} ({len(xyz_coords)}) "
                  f"does not match template molecule ({mol.GetNumAtoms()}). Skipping this frame.")
            continue

        # Create a new Conformer object
        conf = Chem.Conformer(mol.GetNumAtoms())

        # Set the coordinates for each atom in the conformer
        for i, (symbol, x, y, z) in enumerate(xyz_coords):
            # RDKit atoms are ordered internally. We assume the XYZ in the log
            # is in the same order as the atoms in the mol_template.
            # If not, you'd need to map based on atom symbols or indices.
            conf.SetAtomPosition(i, Point3D(x, y, z))

        # Add the conformer to the molecule
        conf_id = mol.AddConformer(conf, assignId=True)

        # Set properties on the conformer object
        # RDKit stores properties as strings, so convert floats
        mol.GetConformer(conf_id).SetDoubleProp('step', float(step_idx + 1))
        mol.GetConformer(conf_id).SetDoubleProp('conf_energy', energy)
        mol.GetConformer(conf_id).SetDoubleProp('gradient_norm', gnorm)
        # Note: Using underscore for 'Gradient_Norm' as properties can sometimes be tricky with spaces

    return mol


temp_dir = '/tmp/temp_geom_opt_f3dr2ifl'
traj_filepath = os.path.join(temp_dir, 'xtbopt.log')

# Now, read the dummy file
quinine_opt = add_xtb_conformers_to_mol(quinine_rdkit, traj_filepath)

print(quinine_opt.GetNumConformers())


In [ ]:
quinine_crest = conformer_search(quinine_xtb, gfn_xtb=2)
print(min([conf.GetDoubleProp('conf_energy')for conf in quinine_crest.GetConformers()]))

quinine_crest_filepath = os.path.join(PATH, 'data', 'quinine_crest.sdf')
save_multiconf_sdf(quinine_crest, quinine_crest_filepath)


In [ ]:
from confworks.util import get_boltzmann_weights

quinine_crest_filepath = os.path.join(PATH, 'data', 'quinine_crest.sdf')
quinine_crest = read_multiconf_sdf(quinine_crest_filepath)

print(get_boltzmann_weights(quinine_crest))

In [ ]:
quinine_combined = Chem.Mol(quinine_rdkit)
for conf in quinine_combined.GetConformers():
    conf.SetProp('level', 'ETKDG')
n_confs= len(quinine_combined.GetConformers())
for conf in quinine_xtb.GetConformers():
    conf.SetProp('level', 'xtb')
    conf = Chem.Conformer(conf)
    
    quinine_combined.AddConformer(conf, assignId=True)

for conf in quinine_crest.GetConformers():
    conf = Chem.Conformer(conf)
    conf.SetProp('level', 'crest')
    quinine_combined.AddConformer(conf, assignId=True)
    
quinine_combined_filepath = os.path.join(PATH, 'data', 'quinine_combined.sdf')
save_multiconf_sdf(quinine_combined, quinine_combined_filepath)

## Calculate a matrix of RMSD values between all conformers

In [ ]:
# TODO sort conformers

In [ ]:
rmsds_matrix = calc_rmsd_matrix_single_mol(quinine_combined)
lower_triangle_indices = np.tril_indices_from(rmsds_matrix, k=-1)
rmsds = rmsds_matrix[lower_triangle_indices]

In [ ]:
# --- 2. Plotting the heatmap ---
fig, ax = plt.subplots(figsize=(5, 5))

# Use imshow to create the heatmap
# cmap='magma' sets the color scheme
im = ax.imshow(rmsds_matrix, cmap='magma')

# --- 3. Adding a color bar ---
# This shows the mapping of colors to values
cbar = ax.figure.colorbar(im, ax=ax, shrink=0.8)
cbar.ax.set_ylabel("Distance", rotation=-90, va="bottom")


# --- 5. Final Touches ---
ax.set_title(f"Conformer RMSD Heatmap")

tick_spacing = 25
ax.set_xticks(np.arange(0, len(rmsds_matrix), tick_spacing))
ax.set_yticks(np.arange(0, len(rmsds_matrix), tick_spacing))

ax.set_xlabel("Index")
ax.set_ylabel("Index")

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# --- Your data ---
# This is a sample list of float values.
# Replace this with your actual data.
# --- Plotting ---
fig, ax = plt.subplots(figsize=(5, 3))

# Plot the density histogram
ax.hist(rmsds, bins=30, alpha=0.7, label='_nolegend_')

# --- Calculate and plot statistics ---
mean = np.mean(rmsds)
std = np.std(rmsds)

# --- Plot Mean Line and Standard Deviation Box ---

# Add a semi-transparent background box for +- 1 standard deviation
ax.axvspan(mean - std, mean + std, alpha=0.3, color='orange', label=f'±Std Dev Range')

# Plot vertical line for the mean on top of the box
ax.axvline(mean, color='r', linestyle='--', linewidth=2, label=f'Mean: {mean:.2f}')


# --- Final Touches ---
ax.set_title('Distribution with Mean and Standard Deviation Range')
ax.set_xlabel('Value')
ax.set_ylabel('Density')
ax.legend() # Display the legend
ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
rmsds_matrix_filepath = os.path.join(PATH, 'data', 'rmsds_matrix.npy')
np.save(rmsds_matrix_filepath, rmsds_matrix)

## Using the RMSD matrix embed all conformers into 2D TSNE

In [ ]:
rmsds_matrix_filepath = os.path.join(PATH, 'data', 'rmsds_matrix.npy')
rmsds_matrix = np.load(rmsds_matrix_filepath)

In [ ]:
energies = [conf.GetDoubleProp('conf_energy') for conf in quinine_combined.GetConformers()]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from openTSNE import TSNE

# --- 2. Fit openTSNE ---
# Initialize TSNE with metric="precomputed". This is the crucial step.
# You can also adjust other parameters like perplexity, n_iter, etc.
print("Fitting t-SNE model...")
tsne = TSNE(
    perplexity=30,
    metric="precomputed",
    n_jobs=-1, # Use all available CPU cores
    random_state=42,
    verbose=True
)

# Fit the model on the distance matrix to get the 2D embedding
embedding = tsne.fit(rmsds_matrix)
print("Fitting complete.")

xs, ys = embedding[:, 0], embedding[:, 1]
zs = energies

# --- 3. Plot the Results ---
print("Plotting the embedding...")
fig, ax = plt.subplots(figsize=(8, 8))
scatter = ax.scatter(xs, ys, c=zs, s=50, alpha=0.7, cmap='viridis')

cbar = fig.colorbar(scatter, ax=ax)
cbar.set_label('Z Values (e.g., Energy, Time, etc.)')

ax.set_title('openTSNE Plot from Distance Matrix')
ax.set_xlabel('t-SNE Dimension 1')
ax.set_ylabel('t-SNE Dimension 2')
ax.grid(True)
plt.show()

## Plot conformers into 3D space, where x and y coordinates are TSNE embeddings, and z coordinate is relative energies of each conformer (dE)

In [ ]:
# This import registers the 3D projection, but is not used directly.
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm

# 1. PREPARE YOUR DATA
# Replace these with your own lists of coordinates
# For this example, we'll create a simple helix.

# 2. CREATE THE 3D PLOT
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(projection='3d')


# 3. PLOT THE DATA
surf = ax.plot_trisurf(xs, ys, zs, cmap=cm.magma, linewidth=0.1, antialiased=True)

# The scatter plot is commented out for a cleaner look, but you can re-enable it.
ax.scatter(xs, ys, zs, c=zs, edgecolors='black', linewidths=1, s=10, cmap='magma')


# 4. ADD A COLOR BAR
# The color bar shows the mapping from Z-values to the 'magma' colormap.
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label='Z Coordinate')

# 4. CUSTOMIZE THE PLOT
ax.set_title("My 3D Scatter Plot")
ax.set_xlabel("X Coordinate")
ax.set_ylabel("Y Coordinate")
ax.set_zlabel("Z Coordinate")

# 5. SHOW THE PLOT
plt.tight_layout()
plt.show()

## Lets find global and local minimas on the energy landscape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.tri import Triangulation

def find_minima_on_mesh(vertices, triangles):
    """
    Finds the global minimum point and all local minima points on a mesh.

    Args:
        vertices (list): A list of vertex dictionaries, e.g., [{'x': x1, 'y': y1, 'z': z1}, ...].
        triangles (list): A list of triangles, where each triangle is a list/tuple
                          of 3 integer indices referring to vertices.

    Returns:
        tuple: A tuple containing:
            - dict: The global minimum point as {'x': x, 'y': y, 'z': z}.
            - list: A list of local minimum points, each as {'x': x, 'y': y, 'z': z}.
    """
    if not vertices:
        return None, []

    num_vertices = len(vertices)

    # 1. Compute Vertex Neighbors
    vertex_neighbors = {i: set() for i in range(num_vertices)}

    for tri in triangles:
        v0, v1, v2 = tri[0], tri[1], tri[2]

        # Add mutual neighbors
        vertex_neighbors[v0].add(v1)
        vertex_neighbors[v0].add(v2)

        vertex_neighbors[v1].add(v0)
        vertex_neighbors[v1].add(v2)

        vertex_neighbors[v2].add(v0)
        vertex_neighbors[v2].add(v1)

    # 2. Find the Global Minimum
    global_min_z = float('inf')
    global_minimum_point = None

    for i, vert in enumerate(vertices):
        if vert['z'] < global_min_z:
            global_min_z = vert['z']
            global_minimum_point = vert

    # 3. Find Local Minima
    local_minima_points = []

    for i in range(num_vertices):
        current_vertex_z = vertices[i]['z']
        is_local_minimum = True

        # Check if any neighbor has a strictly lower Z value
        for neighbor_idx in vertex_neighbors[i]:
            if vertices[neighbor_idx]['z'] < current_vertex_z:
                is_local_minimum = False
                break # Not a local minimum, move to the next vertex

        if is_local_minimum:
            local_minima_points.append(vertices[i])

    return global_minimum_point, local_minima_points

# --- How to get the mesh/triangles from xs, ys, zs ---

# 1. Prepare your vertices list of dictionaries
vertices = [{'x': xs[i], 'y': ys[i], 'z': zs[i]} for i in range(len(xs))]

# 2. Use Triangulation to get the triangles (simplices)
# This performs a Delaunay triangulation on the (x, y) points.
triang = Triangulation(xs, ys)
triangles = triang.get_masked_triangles() # Corrected line # These are the indices of vertices forming each triangle

# 3. Call the find_minima_on_mesh function
global_min, local_mins = find_minima_on_mesh(vertices, triangles)

print("--- Analysis Results ---")
print(f"Global Minimum Point: {global_min}")
print(f"Local Minima Points: {local_mins}")

# --- Optional: Visualize the found minima on the plot ---
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the original surface
surf = ax.plot_trisurf(xs, ys, zs, cmap=cm.magma, linewidth=0.1, antialiased=True, alpha=0.8)

# Mark all local minima
if local_mins:
    local_min_x = [p['x'] for p in local_mins]
    local_min_y = [p['y'] for p in local_mins]
    local_min_z = [p['z'] for p in local_mins]
    ax.scatter(local_min_x, local_min_y, local_min_z,
               color='blue', s=50, label='Local Minima', edgecolors='black', linewidth=0.5)
    

# Mark the global minimum
if global_min:
    ax.scatter(global_min['x'], global_min['y'], global_min['z'],
               color='green', s=100, label='Global Minimum', edgecolors='black', linewidth=1)

ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.set_title('Mesh with Global and Local Minima')
fig.colorbar(surf, shrink=0.5, aspect=10)
plt.legend()
plt.show()

## Now lets optimize all these 300 conformers to get better better minimas

In [ ]:
## Optimize all conformers using xtb, sort out conformers based on RMSD
## plot all original conformers and now newly optimized conformers with all their steps on a single 3D plot landscape 
# (consider adding xtb optimization trails for conformers)

## plot all of that on TSNE (projection from above)
## plot all of that bananas

In [ ]:
ref_mol = Chem.Mol(quinine_3D)
combined_mol = Chem.Mol(ref_mol)

mols_list = [quinine_3D,]*quinine_3D.GetNumConformers()
for i, mol in enumerate(tqdm(mols_list)):
    if i == 0: 
        ref_mol.RemoveAllConformers()
        conf = quinine_3D.GetConformer(i)
        ref_mol.AddConformer(conf)
        continue

    mol.RemoveAllConformers()
    conf = quinine_3D.GetConformer(i)
    mol.AddConformer(conf)
    
    # Align each molecule to the reference
    Chem.rdMolAlign.AlignMol(mol, ref_mol, prbCid=-1, refCid=-1)

    combined_mol = Chem.CombineMols(combined_mol, mol)
    
# Visualize in 3D viewer
IPythonConsole.drawMol3D(combined_mol)


## Here we will visualize our conformational space using *Chemiscope* and *STK*

In [ ]:
import itertools as it

import stk
import stko

import chemiscope

In [ ]:
quinine_combined = trj_list[2]

energies = [conf.GetDoubleProp('conf_energy') for conf in quinine_combined.GetConformers()]
print(len(energies))
print(min(energies))

In [ ]:
Chem.rdMolAlign.AlignMolConformers(quinine_combined)

mol_list = []
Chem.Kekulize(quinine_combined)
for conf in quinine_combined.GetConformers():
    mol_copy = Chem.Mol(quinine_combined)
    mol_copy.RemoveAllConformers()
    mol_copy.AddConformer(conf, assignId=True)
    mol_list.append(mol_copy)

structures = [stk.BuildingBlock.init_from_rdkit_mol(mol) for mol in mol_list]

properties = {
    "index": np.arange(len(structures)),
    # an example of the verbose definition
    "xtb_energy": {
        "target": "structure",
        "values": energies,
        "units": "Eh",
        "description": "electronic energy computed with GFN2-xTB",
    },
}

In [ ]:
chemiscope.show(
    frames=structures,
    properties=properties,
    settings=chemiscope.quick_settings(
        x="index",
        y="xtb_energy",
        color="xtb_energy",
        structure_settings={
            "atoms": True,
            "bonds": True,
            "spaceFilling": False,
        },
    ),
)

In [ ]:
from openTSNE import TSNE

def rmsd_tsne(frames, environments):
    if environments is not None:
        raise ValueError("'environments' are not supported by this featurizer")

    tsne = TSNE(
    perplexity=30,
    metric="precomputed",
    n_jobs=-1, # Use all available CPU cores
    random_state=42,
    verbose=False
    )

    # this function doesn't use frames
    # but instead uses predefined rmsds_matrix
    # which is used for tsne coordinates calculation
    return tsne.fit(rmsds_matrix)

embedding = rmsd_tsne(structures, None)
print("Fitting complete.")

tsne1, tsne2 = embedding[:, 0], embedding[:, 1]

properties['tSNE-1'] = {
    "target": "structure",
    "values": tsne1,
}
properties['tSNE-2'] = {
    "target": "structure",
    "values": tsne2,
}


In [ ]:
cs = chemiscope.explore(
    structures, 
    featurize=rmsd_tsne, 
    properties=properties,
    settings=chemiscope.quick_settings(
        x="tSNE-1",
        y="tSNE-2",
        color="xtb_energy",
        structure_settings={
            "atoms": True,
            "bonds": True,
            "spaceFilling": False,
        },
    ),
)

In [ ]:
cs

In [ ]:
chemiscope.write_input(
    path="data/quinine_conf_landscape.json.gz",
    frames=structures,
    properties=properties,
    meta={
        "name": "Conformational landscape of quinine.",
        "description": (
            "This Chemiscope visualization contains 300 conformers of quinine molecule,"
            "which were embedded into tSNE based on RMSD between all conformer pairs," 
            "and color mapped according to their single point GFN2-xTB energies."
        ),
        "authors": ["Galymzhan Moldagulov (PhD candidate @ IBS-CARS & UNIST, KR)"],
        "references": [
            (
                "TBA"
            )
        ],

    },
    settings=chemiscope.quick_settings(
        x="tSNE-1",
        y="tSNE-2",
        color="xtb_energy",
        structure_settings={
            "atoms": True,
            "bonds": True,
            "spaceFilling": False,
        },
    ),
)